In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator

from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
%cd drive/MyDrive/intestine/

/content/drive/MyDrive/intestine


In [ ]:
!unzip p2.zip

Streaming output truncated to the last 5000 lines.
  inflating: kvasir-dataset-v2/normal-cecum/00b1b20b-ce2b-449c-93fc-6df854e3ab33.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/00b99f19-2c31-4c7c-931a-4c3b38d70d1a.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/00f3d2cc-93ea-40f0-9b88-b159b07a49cb.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/0111fb06-3113-4abf-9050-7bfb8d4688f5.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/0163b3a2-9aa1-4b23-a9a8-aff07a738f16.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/0191eafd-0367-4772-9181-296b80dbf917.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/01af3454-037f-4708-b73c-6ec4423b6a61.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/01b9de1c-1481-4056-b27a-4d8fa09da06f.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/01bddb73-9c5d-4ac0-8b25-cc7524b7d191.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/01c48bf5-3f79-45dd-9390-add72a6da946.jpg  
  inflating: kvasir-dataset-v2/normal-cecum/01c49663-9674-44ae-b61e-05243a30e4a

In [ ]:
for i in os.listdir("kvasir-dataset-v2"):
  print(i)

dyed-lifted-polyps
dyed-resection-margins
esophagitis
normal-cecum
normal-pylorus
normal-z-line
polyps
ulcerative-colitis


In [ ]:
os.listdir("kvasir-dataset-v2")

['dyed-lifted-polyps',
 'dyed-resection-margins',
 'esophagitis',
 'normal-cecum',
 'normal-pylorus',
 'normal-z-line',
 'polyps',
 'ulcerative-colitis']

In [ ]:
!ls kvasir-dataset-v2/

dyed-lifted-polyps	esophagitis   normal-pylorus  polyps
dyed-resection-margins	normal-cecum  normal-z-line   ulcerative-colitis


In [ ]:
for i in os.listdir("kvasir-dataset-v2/"):
 print(i,len(os.listdir("kvasir-dataset-v2/"+i)))

dyed-lifted-polyps 1000
dyed-resection-margins 1000
esophagitis 1000
normal-cecum 1000
normal-pylorus 1000
normal-z-line 1000
polyps 1000
ulcerative-colitis 1000


In [ ]:

try:
 os.mkdir("train")
 os.mkdir("test")
except:
  pass
for i in os.listdir("kvasir-dataset-v2/"):
  try:
   os.mkdir("train/"+i)
   os.mkdir("test/"+i)
  except:
   pass
  for j in os.listdir("kvasir-dataset-v2/"+i)[:700]:
    os.rename("kvasir-dataset-v2/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("kvasir-dataset-v2/"+i)[:300]:
    os.rename("kvasir-dataset-v2/"+i+"/"+j,"test/"+i+"/"+j)

In [ ]:
!ls test

dyed-lifted-polyps	esophagitis   normal-pylorus  polyps
dyed-resection-margins	normal-cecum  normal-z-line   ulcerative-colitis


In [ ]:
def img_Data(dir_path,target_size,batch,class_1st,preporcssing):
  if preporcssing:
    gen_object =ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes=class_1st,
                                        shuffle=True))

In [ ]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 5600 images belonging to 8 classes.
Found 2400 images belonging to 8 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax')

In [ ]:
base_model.trainable = False

In [ ]:
from keras.src.layers.serialization import activation
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint('.md1_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 1024)              1311744   
                                                                 
 dense_4 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3579978 (13.66 MB)
Trainable params: 1321994 (5.04 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [1]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=3)

NameError: ignored